## This is the main script where all simulations are launched

In [31]:
import random
import time
import warnings

random.seed(271)

# Given
n = 1000
c_t = 500
c_s = 1
v_w = 50
v_l = 0

# Searched
triangle = [max(n*0.15-x,0) for x in range(n)]
w_distribution = {x: triangle[x]/sum(triangle) for x in range(n)}

# Technical params
proba_threshold = 0.00002
rep_n = 1000
print_progress_every_x_sec = 10
break_exact_method_after_x_sec = 5

events = {}

def calc_stats(obs):
    stat_dict = {}
    stat_dict["mean"] = sum([payout*proba for payout,proba in obs])
    stat_dict["proba_win"] = sum([proba for payout,proba in obs if payout>0])
    stat_dict["proba_loss"] = sum([proba for payout,proba in obs if payout<=0])
    if stat_dict["proba_win"]>0:
        stat_dict["mean_win"] = sum([payout*proba for payout,proba in obs if payout>0])/sum([proba for payout,proba in obs if payout>0])
    else:
        stat_dict["mean_win"] = None

    if stat_dict["proba_loss"]>0:
        stat_dict["mean_loss"] = sum([payout*proba for payout,proba in obs if payout<=0])/sum([proba for payout,proba in obs if payout<=0])
    else:
        stat_dict["mean_loss"] = None
    return stat_dict


def calc_metastats(payouts):
    stat_dict = {"mean": 0, "mean_win": 0, "mean_loss": 0, "proba_win": 0, "proba_loss": 0}
    for w, w_proba in w_distribution.items():
        stat_dict["mean"] += payouts[w]["stats"]["mean"]*w_proba
        stat_dict["proba_win"] += payouts[w]["stats"]["proba_win"]*w_proba
        if payouts[w]["stats"]["proba_win"] >0:
            stat_dict["mean_win"] += payouts[w]["stats"]["mean_win"]*payouts[w]["stats"]["proba_win"]*w_proba
        stat_dict["proba_loss"] += payouts[w]["stats"]["proba_loss"]*w_proba
        if payouts[w]["stats"]["proba_loss"] >0:
            stat_dict["mean_loss"] += payouts[w]["stats"]["mean_loss"]*payouts[w]["stats"]["proba_loss"]*w_proba
    return stat_dict


def calc_payout(decision, hist_list, w):
    l = n-w
    if len(hist_list) == n:
        warnings.warn("Function never stopped, it bought the whole box as singles.")
        return w*v_w + l*v_l - len(hist_list)*c_s
    elif decision == "buy":
        return w*v_w + l*v_l - c_t - len(hist_list)*c_s
    elif decision == "refuse":
        return sum(hist_list)*v_w + (len(hist_list) - sum(hist_list))*v_l - len(hist_list)*c_s
    else:
        raise ValueError("Wrong return value from function ! Should be [buy, refuse, continue]")

            
def run_random(f):
    payouts = {}
    for w, w_proba in w_distribution.items():
        payouts[w] = {"obs": []}
        for i_rep in range(rep_n):

            hist_list = []
            while True:
                if "random_float" in f.__code__.co_varnames:
                    decision = f(hist_list, random_float=random.uniform(0, 1))
                else:
                    decision = f(hist_list)
                if decision == "continue" and len(hist_list) < n:
                    hist_list += random.choices([1,0], cum_weights=[w-sum(hist_list), n-len(hist_list)])
                    continue
                else:
                    payout = calc_payout(decision, hist_list, w)
                    payouts[w]["obs"] += [(payout, 1/rep_n)]
                    break
                    
        payouts[w]["stats"] = calc_stats(payouts[w]["obs"])

    payouts["stats"] = calc_metastats(payouts)
    return payouts


def evaluate_exact(w, w_distribution, start_time):
    w_proba = w_distribution[w]
    payout_list = []
    tree = [((),1)]
    while len(tree) > 0:
        if time.time() - start_time > break_exact_method_after_x_sec:
            raise TimeoutError

        hist_list, proba = tree.pop()
        n_tries = len(hist_list)

        if "random_float" in f.__code__.co_varnames:
            # Case if decision is partly random
            if proba > proba_threshold:
                n_randoms = 5
                proba /= n_randoms
                decision_list = [f(hist_list, random_float=random.uniform(0, 1)) for random_int in range(n_randoms)]
            else:
                decision_list = [f(hist_list, random_float=random.uniform(0, 1))]
        else:
            # Case if decision is deterministic
            decision_list = [f(hist_list)]

        for decision in decision_list:
            if decision == "continue" and n_tries < n:
                proba_win_marginal = (w-sum(hist_list))/(n-n_tries)
                proba_loss_marginal = 1 - proba_win_marginal

                if proba > proba_threshold:
                    if proba_win_marginal > 0:
                        tree.append((hist_list + (1,), proba*proba_win_marginal))
                    if proba_loss_marginal > 0:
                        tree.append((hist_list + (0,), proba*proba_loss_marginal))
                else:
                    # Case if probability gets too small to be relevant
                    # Number of leafs of tree must be kept reasonable -> stop in smallest branches
                    random_draw = random.choices([1,0], cum_weights=[proba_win_marginal, 1])[0]
                    tree.append((hist_list + (random_draw,), proba))

            else:
                payout = calc_payout(decision, hist_list, w)
                payout_list += [(payout, proba)]
                
        return payout_list


def run_exact(f):
    payouts = {}
    start_time = time.time()
    current_time = start_time
    w_n = len(w_distribution)
    try:
        for iw, w in enumerate(w_distribution):
            if time.time() - current_time > print_progress_every_x_sec:
                current_time = time.time()
                remaining_est_time = (current_time-start_time)/iw*(w_n-iw)
                print("Est. time remaining: " + str(round(remaining_est_time)) + "s")
            
            payouts[w]["obs"] = evaluate_exact(w, w_distribution, start_time)
            payouts[w]["stats"] = calc_stats(payouts[w]["obs"])

        payouts["stats"] = calc_metastats(payouts)
    except TimeoutError:
        warnings.warn("Function takes too long to be exactly evaluated; falling back to Monte Carlo.")
        payouts = run_random(f)
    return payouts



In [32]:
def buy(hist_list):
    return("buy")


def buy_if_x_success(hist_list, x):
    if sum(hist_list) == x:
        return("buy")
    return("continue")


def buy_if_extrapolation_fix_sample_x_yields_positive_profit(hist_list, x):
    if len(hist_list) == x:
        if (sum(hist_list)*v_w + (len(hist_list)-sum(hist_list))*v_l)/x*n > c_t + len(hist_list)*c_s:
            return("buy")
        else:
            return("refuse")
    return("continue")


def buy_if_success_ratio_geq_x(hist_list, x):
    if len(hist_list) > 0 and sum(hist_list)/len(hist_list) >= x:
        return("buy")
    return("continue")


def buy_random(hist_list, random_float):
    if random_float <= 0.5:
        return("buy")
    return("continue")
    

def buy_refuse_random(hist_list, random_float):
    if random_float < 0.34:
        return("buy")
    elif random_float < 0.67:
        return("refuse")
    return("continue")


competition = {"buy": buy, 
           "buy_if_x_success": buy_if_x_success, 
           "buy_if_extrapolation_fix_sample_x_yields_positive_profit": buy_if_extrapolation_fix_sample_x_yields_positive_profit, 
           "buy_random": buy_random,
           "buy_refuse_random": buy_refuse_random}

In [33]:
#from scipy.optimize import minimize 
#minimize(f, x0, args=(a, b, c))

from scipy import optimize
import functools

#buy_if_success_ratio_geq_x_set = functools.partial(buy_if_success_ratio_geq_x, x=0.02)
def buy_if_success_ratio_geq_x_set(x): return buy_if_success_ratio_geq_x(x, 0.02)


run_exact(buy_if_success_ratio_geq_x_set)["stats"]

#minimum = optimize.fmin(f, 1)


/home/andarin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: Function never stopped, it bought the whole box as singles.
/home/andarin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:151: UserWarning: Function takes too long to be exactly evaluated; falling back to Monte Carlo.


{'mean': 1887.8035127593812,
 'mean_win': 1988.2971898454741,
 'mean_loss': -100.49367708609276,
 'proba_win': 0.8234514790286984,
 'proba_loss': 0.17654852097130258}

In [2]:
results = {k: run_random(v) for k,v in competition.items()}
{fname: v["stats"] for fname,v in results.items()}

{'buy_0': {'mean': 1983.3333333333342,
  'mean_win': 2019.0286975717447,
  'mean_loss': -35.695364238410505,
  'proba_win': 0.8591611479028705,
  'proba_loss': 0.14083885209713032},
 'buy_1_1': {'mean': 157.19581456953756,
  'mean_win': 158.26200724061803,
  'mean_loss': -1.0661926710816723,
  'proba_win': 0.04790004415011041,
  'proba_loss': 0.9520999558498904},
 'buy_projection_fix_sample': {'mean': 1082.3230816777013,
  'mean_win': 1089.8390852097127,
  'mean_loss': -7.516003532008771,
  'proba_win': 0.35631708609271534,
  'proba_loss': 0.6436829139072849},
 'buy_random': {'mean': 1982.3321180573928,
  'mean_win': 2018.167963885207,
  'mean_loss': -35.83584582781448,
  'proba_win': 0.8591611479028705,
  'proba_loss': 0.14083885209713032},
 'buy_refuse_random': {'mean': 1004.2797159381904,
  'mean_win': 1022.1814852980131,
  'mean_loss': -17.901769359823426,
  'proba_win': 0.4456454746136871,
  'proba_loss': 0.5543545253863141}}

In [12]:
competition = {
           "buy_random": buy_random,
           "buy_refuse_random": buy_refuse_random}

for fname,f in competition.items():
    results = run_exact(f)
    print(fname)
    print(results["stats"])
    #break


Est. time remaining: 29s
Est. time remaining: 28s
Est. time remaining: 25s
Est. time remaining: 19s
Est. time remaining: 7s
buy_random
{'mean': 1982.3327642328682, 'mean_win': 2018.193724354025, 'mean_loss': -35.86096012115674, 'proba_win': 0.859161147902875, 'proba_loss': 0.14083885209712935}
buy_refuse_random
{'mean': 1017.3619907223801, 'mean_win': 1039.8165768092467, 'mean_loss': -22.45458608686737, 'proba_win': 0.47744906391943454, 'proba_loss': 0.522550936080566}


In [49]:
print(dir(buy_if_success_ratio_geq_x_set))
import inspect

print(inspect.currentframe().f_code.co_name)


['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
<module>


In [14]:
a = [1,2]
a.append(3)
a.pop()

3